In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import pickle

from src.paths import MODEL_DIR, PREPROCESSED_DATA_DIR, RAW_DATA_DIR, SUBMISSION_DIR

In [14]:
with open(MODEL_DIR / 'best_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [15]:
data_test = pd.read_pickle(PREPROCESSED_DATA_DIR / 'data_test.pkl')
data_test.head()

,Id,Latitude,Longitude,Week,Dayofyear,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Tavg_1,PrecipTotal_1,Tmin_2,Tavg_2,PrecipTotal_2,DewPoint_2_mean_l4_w1
0,3634,41.954690,-87.800991,27,183,0.0,1.0,0.0,71,0.0,57,71,0.0,67.0
1,3635,41.954690,-87.800991,27,183,0.0,0.0,1.0,71,0.0,57,71,0.0,67.0
2,3636,41.954690,-87.800991,27,183,1.0,0.0,0.0,71,0.0,57,71,0.0,67.0
3,3650,41.974089,-87.824812,27,183,0.0,1.0,0.0,71,0.0,57,71,0.0,67.0
4,3651,41.974089,-87.824812,27,183,0.0,0.0,1.0,71,0.0,57,71,0.0,67.0


In [16]:
predictions = model.predict_proba(data_test.drop(['Id'], axis=1))[:,1]
data_test['proba'] = predictions
data_test.head()

,Id,Latitude,Longitude,Week,Dayofyear,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Tavg_1,PrecipTotal_1,Tmin_2,Tavg_2,PrecipTotal_2,DewPoint_2_mean_l4_w1,proba
0,3634,41.954690,-87.800991,27,183,0.0,1.0,0.0,71,0.0,57,71,0.0,67.0,0.073839
1,3635,41.954690,-87.800991,27,183,0.0,0.0,1.0,71,0.0,57,71,0.0,67.0,0.073249
2,3636,41.954690,-87.800991,27,183,1.0,0.0,0.0,71,0.0,57,71,0.0,67.0,0.073222
3,3650,41.974089,-87.824812,27,183,0.0,1.0,0.0,71,0.0,57,71,0.0,67.0,0.073461
4,3651,41.974089,-87.824812,27,183,0.0,0.0,1.0,71,0.0,57,71,0.0,67.0,0.072873


In [17]:
sample = pd.read_csv(RAW_DATA_DIR / 'sampleSubmission.csv')
predictions = data_test[['Id', 'proba']]
predictions
results = pd.merge(sample, predictions, on='Id', how='outer').fillna(0)
results['WnvPresent']=results['proba']
results.drop('proba', axis=1, inplace=True)
results

,Id,WnvPresent
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
...,...,...
116288,116289,0.0
116289,116290,0.0
116290,116291,0.0
116291,116292,0.0


In [18]:
results.to_csv(SUBMISSION_DIR / 'test_results.csv', index=False)

In [19]:
!kaggle competitions submit -c predict-west-nile-virus -f /home/aga/repos/west_nile_virus/data/submission/test_results.csv -m test

100%|██████████████████████████████████████| 1.22M/1.22M [00:01<00:00, 1.05MB/s]
Successfully submitted to West Nile Virus Prediction

In [20]:
s = !kaggle competitions submissions -c predict-west-nile-virus

In [21]:
public_score = s[3].split()[-2]
private_score = s[3].split()[-1]
print(f"Public score: {public_score}\nPrivate score: {private_score}")

Public score: 0.72095
Private score: 0.67003
